In [1]:
import pandas as pd
from datetime import datetime
import pickle
import os

In [2]:
with open('semanas.pkl', 'rb') as archivo:
    variable_cargada = pickle.load(archivo)
    
variable_cargada["307"] = "SEMANA 42/2023 (del 09/10/2023 al 15/10/2023)"

In [10]:
variable_cargada

{'306': 'SEMANA 41/2023 (del 02/10/2023 al 08/10/2023)',
 '305': 'SEMANA 40/2023 (del 25/09/2023 al 01/10/2023)',
 '304': 'SEMANA 39/2023 (del 18/09/2023 al 24/09/2023)',
 '303': 'SEMANA 38/2023 (del 11/09/2023 al 17/09/2023)',
 '302': 'SEMANA 37/2023 (del 04/09/2023 al 10/09/2023)',
 '301': 'SEMANA 36/2023 (del 28/08/2023 al 03/09/2023)',
 '300': 'SEMANA 35/2023 (del 21/08/2023 al 27/08/2023)',
 '299': 'SEMANA 34/2023 (del 14/08/2023 al 20/08/2023)',
 '298': 'SEMANA 33/2023 (del 07/08/2023 al 13/08/2023)',
 '297': 'SEMANA 32/2023 (del 31/07/2023 al 06/08/2023)',
 '296': 'SEMANA 31/2023 (del 24/07/2023 al 30/07/2023)',
 '295': 'SEMANA 30/2023 (del 17/07/2023 al 23/07/2023)',
 '294': 'SEMANA 29/2023 (del 10/07/2023 al 16/07/2023)',
 '293': 'SEMANA 28/2023 (del 03/07/2023 al 09/07/2023)',
 '292': 'SEMANA 27/2023 (del 26/06/2023 al 02/07/2023)',
 '291': 'SEMANA 26/2023 (del 19/06/2023 al 25/06/2023)',
 '290': 'SEMANA 25/2023 (del 12/06/2023 al 18/06/2023)',
 '289': 'SEMANA 24/2023 (del 05

In [7]:
carabinero = pd.read_excel(r"C:\Users\limc_\carabineros\carabineros.xlsx")
carabinero= carabinero[carabinero["ZONA"] == 'METROPOLITANA']
import math
def umbrales(fila):   #df2.iloc[1]
    promedio = fila.anno1 * 1 + fila.anno2 * 2 + fila.anno3 * 3 + fila.anno4 * 4 + fila.anno5 * 5
    promedio = promedio / 15
    sigma_cuadrado = ((fila.anno1 -promedio) ** 2)* 1 + ((fila.anno2 -promedio) ** 2) * 2 + ((fila.anno3 -promedio) ** 2) * 3 + ((fila.anno4 -promedio) ** 2) * 4 + ((fila.anno5 -promedio) ** 2) * 5 
    sigma_cuadrado = sigma_cuadrado / 15
    sigma = math.sqrt(sigma_cuadrado)
    if(sigma == 0):
        return 0
    return round((fila.anno6 - promedio) / sigma,2)


def Transformar(texto):
    try:
        return int(texto)
    except:
        return 0

In [9]:
# Ruta de la carpeta que deseas recorrer
carpeta = 'umbrales'

# Lista para almacenar los nombres de los archivos Excel encontrados
archivos_excel = []

# Recorre la carpeta
for filename in os.listdir(carpeta):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        archivos_excel.append(os.path.join(carpeta, filename))
        
acumulador = []

# Ahora, puedes leer los archivos Excel
for archivo_excel in archivos_excel:
    try:
        print(archivo_excel)
        df = pd.read_excel(archivo_excel)
        del df['Unnamed: 0']
        #del df['index']
        #df.columns = columnas
        #df = df.drop(index=[0, 1]).reset_index(drop=True)
        acumulador.append(df.copy())
        #df
        # Haz algo con el DataFrame, por ejemplo, imprimir las primeras filas
        #print(f'Contenido de {archivo_excel}:')
        #print(df.head())
    except Exception as e:
        print(f'Error al leer {archivo_excel}: {str(e)}')
        
df = pd.concat(acumulador)
df2 = df
df2["nombre"] = df2["Umbrales"]
df2["anno1"] = df2["2018"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno2"] = df2["2019"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno3"] = df2["2020"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno4"] = df2["2021"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno5"] = df2["2022"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno6"] = df2["2023"].fillna(0)  #.str.replace(r'^-$', '0', regex=True).fillna(0)
df2["anno1"]=df2["anno1"].apply(Transformar)
df2["anno2"]=df2["anno2"].apply(Transformar)
df2["anno3"]=df2["anno3"].apply(Transformar)
df2["anno4"]=df2["anno4"].apply(Transformar)
df2["anno5"]=df2["anno5"].apply(Transformar)
df2["anno6"]=df2["anno6"].apply(Transformar)
df2["umbral"] = df2.apply(umbrales,axis=1)
#df2["Delito"] = df2["Tipo"]
df2["semana"] = df2["semana"].fillna(0).round().astype(int).astype(str)
df2["semana"] = df2["semana"].apply(lambda x:variable_cargada[x])
df2["FechaInicial"] = df2["semana"].apply(lambda x:datetime.strptime(x.split(" ")[3], "%d/%m/%Y"))
df2["FechaFinal"]   = df2["semana"].apply(lambda x:datetime.strptime(x.split(" ")[5], "%d/%m/%Y)"))

df2 = df2.merge(carabinero,left_on="COD_CUADRANTE_INTERNO",right_on="CUA_CODIGO",how="outer")
df2["actual"] = 2023
df2["anterior"] = 2022
df2["titulo"] = df2["CUA_DESCRI"]

del df2['COD_CUADRANTE_INTERNO']
del df2["2018"]
del df2["2019"]
del df2["2020"]
del df2["2021"]
del df2["2022"]
del df2["2023"]
#del df2["Unnamed: 0"]
del df2["Umbrales"]
del df2["Umbral"]
del df2["Tipo"]
del df2["type"]
del df2["features"]
del df2["TOOLTIP"]
del df2["SUP_KM2"]
df2.to_excel("consolidado_umbrales.xlsx", index=False)

umbrales\avance_umbrales_296.xlsx
umbrales\avance_umbrales_297.xlsx
umbrales\avance_umbrales_298.xlsx
umbrales\avance_umbrales_299.xlsx
umbrales\avance_umbrales_300.xlsx
umbrales\avance_umbrales_301.xlsx
umbrales\avance_umbrales_302.xlsx
umbrales\avance_umbrales_303.xlsx
umbrales\avance_umbrales_304.xlsx
umbrales\avance_umbrales_305.xlsx
umbrales\avance_umbrales_306.xlsx
umbrales\avance_umbrales_307.xlsx


In [5]:
df2.columns

Index(['semana', 'nombre', 'anno1', 'anno2', 'anno3', 'anno4', 'anno5',
       'anno6', 'umbral', 'FechaInicial', 'FechaFinal', 'CUA_CODIGO',
       'CUA_SUP', 'COMUNA', 'UNIDAD', 'DESTACAMEN', 'PREFECTURA', 'ZONA',
       'NUM_CUAD', 'CUA_DESCRI', 'CUA_TIPO', 'CUA_ESTADO', 'CUA_FTERMI',
       'UNI_CODIGO', 'CUA_ANO', 'COD_AUPOL', 'COM_CODIGO', 'SHAPE_LENG',
       'SHAPE_AREA', 'COD_DESTAC', 'COD', 'actual', 'anterior', 'titulo'],
      dtype='object')

In [4]:
df.columns

Index(['Unnamed: 0', 'Umbrales', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Umbral', 'Tipo', 'COD_CUADRANTE_INTERNO', 'semana'],
      dtype='object')